In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda, GRU
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
dataset = pd.read_csv("../data/dowjones/AMZN_2006-01-01_to_2018-01-01.csv", index_col='Date', parse_dates=['Date'])
dataset.head()

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,47.47,47.85,46.25,47.58,7582127,AMZN
2006-01-04,47.48,47.73,46.69,47.25,7440914,AMZN
2006-01-05,47.16,48.20,47.11,47.65,5417258,AMZN
2006-01-06,47.97,48.58,47.32,47.87,6154285,AMZN
2006-01-09,46.55,47.10,46.40,47.08,8945056,AMZN


In [6]:
dataset = dataset[["Close", "Name"]]
dataset = dataset.pivot_table(values='Close', index=dataset.index, columns='Name', aggfunc='first')
dataset.head()

Name,AMZN
Date,
2006-01-03,47.58
2006-01-04,47.25
2006-01-05,47.65
2006-01-06,47.87
2006-01-09,47.08


In [7]:
x_train = dataset.values[:750]
x_train.shape

(750, 1)

In [8]:
x_test = dataset.values[510:1000]
x_test.shape

(490, 1)

In [9]:
i = 0
timestep = 240
train_gen = TimeseriesGenerator(x_train, x_train,
                                length=timestep, sampling_rate=1,
                                batch_size=510)
test_gen = TimeseriesGenerator(x_test, x_test,
                               length=timestep, sampling_rate=1,
                               batch_size=250)

In [10]:
print(f"x train shape: {x_train.shape}")
print(f"x test shape: {x_test.shape}")

x train shape: (750, 1)
x test shape: (490, 1)


In [11]:
# x_train = train_gen[0][0]
# y_train = train_gen[0][1]
# y_test = test_gen[0][0]
# y_test = test_gen[0][1]

In [12]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [14]:
# expected input data shape: (batch_size, timesteps, data_dim)
regressor = Sequential()
# regressor.add(LSTM(units=25, input_shape=(timestep, 1), dropout=0.1))
# regressor.add(GRU(units=25, return_sequences=True, input_shape=(timestep, 1), dropout=0.3, recurrent_dropout=0.3))
# regressor.add(GRU(25, return_sequences=True, dropout=0.3))
# regressor.add(GRU(25, dropout=0.3, recurrent_dropout=0.3))

regressor.add(LSTM(units=100, input_shape=(timestep, 1), dropout=0.1, recurrent_dropout=0.1))
# regressor.add(LSTM(25, return_sequences=True, dropout=0.1))
# regressor.add(LSTM(25, return_sequences=True, dropout=0.1))
# regressor.add(LSTM(50, dropout=0.3, recurrent_dropout=0.3))

# regressor.add(Dense(10,input_shape=(timestep, ), activation='relu'))
# regressor.add(Dense(100, activation='relu'))
# regressor.add(Dense(100, activation='relu'))

# regressor.add(Reshape((31, 2)))
# regressor.add(Lambda(lambda x: softmax(x, axis=-1)))

# regressor.add(Dense(2, activation='softmax'))
regressor.add(Dense(1, activation='relu'))
regressor.compile(loss='mean_squared_error',
                  optimizer='rmsprop',
                  metrics=['mean_squared_error'])
# regressor.compile(loss='binary_crossentropy',
#                   optimizer='rmsprop',
#                   metrics=['accuracy'])
# regressor.fit(x, y, epochs=1000,batch_size=1000, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=20),
#              ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

regressor.fit_generator(train_gen, steps_per_epoch=len(train_gen),
                        epochs=1000, validation_data=test_gen,
                        callbacks=[
                            EarlyStopping(monitor='val_loss',
                                          mode='min', patience=10),
                            ModelCheckpoint(filepath="../model/mymodel.h5",
                                            monitor='val_acc',
                                            save_best_only=True)])


Epoch 1/1000
1/1 [==============================] - 1s 919ms/step - loss: 4912.6489 - mean_squared_error: 4912.6489 - val_loss: 7647.4956 - val_mean_squared_error: 7647.4956
Epoch 2/1000


/home/tqa/anaconda3/envs/projet_S5/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


1/1 [==============================] - 0s 300ms/step - loss: 4854.4404 - mean_squared_error: 4854.4404 - val_loss: 7614.3232 - val_mean_squared_error: 7614.3232
Epoch 3/1000
1/1 [==============================] - 0s 305ms/step - loss: 4810.2266 - mean_squared_error: 4810.2266 - val_loss: 7530.1108 - val_mean_squared_error: 7530.1108
Epoch 4/1000
1/1 [==============================] - 0s 362ms/step - loss: 4750.8701 - mean_squared_error: 4750.8701 - val_loss: 7441.6831 - val_mean_squared_error: 7441.6831
Epoch 5/1000
1/1 [==============================] - 0s 367ms/step - loss: 4707.3184 - mean_squared_error: 4707.3184 - val_loss: 7411.7168 - val_mean_squared_error: 7411.7168
Epoch 6/1000
1/1 [==============================] - 0s 297ms/step - loss: 4685.9536 - mean_squared_error: 4685.9536 - val_loss: 7384.5234 - val_mean_squared_error: 7384.5234
Epoch 7/1000
1/1 [==============================] - 0s 262ms/step - loss: 4668.0176 - mean_squared_error: 4668.0176 - val_loss: 7356.5425 - val

Epoch 49/1000
1/1 [==============================] - 0s 270ms/step - loss: 3610.9602 - mean_squared_error: 3610.9602 - val_loss: 6082.6704 - val_mean_squared_error: 6082.6704
Epoch 50/1000
1/1 [==============================] - 0s 266ms/step - loss: 3600.1033 - mean_squared_error: 3600.1033 - val_loss: 6058.8071 - val_mean_squared_error: 6058.8071
Epoch 51/1000
1/1 [==============================] - 0s 257ms/step - loss: 3580.5156 - mean_squared_error: 3580.5156 - val_loss: 6035.1982 - val_mean_squared_error: 6035.1982
Epoch 52/1000
1/1 [==============================] - 0s 291ms/step - loss: 3562.8416 - mean_squared_error: 3562.8416 - val_loss: 6008.7031 - val_mean_squared_error: 6008.7031
Epoch 53/1000
1/1 [==============================] - 0s 282ms/step - loss: 3546.7036 - mean_squared_error: 3546.7036 - val_loss: 5979.7036 - val_mean_squared_error: 5979.7036
Epoch 54/1000
1/1 [==============================] - 0s 260ms/step - loss: 3525.9946 - mean_squared_error: 3525.9946 - val_lo

Epoch 96/1000
1/1 [==============================] - 0s 263ms/step - loss: 2981.3333 - mean_squared_error: 2981.3333 - val_loss: 5241.3672 - val_mean_squared_error: 5241.3672
Epoch 97/1000
1/1 [==============================] - 0s 302ms/step - loss: 2971.4446 - mean_squared_error: 2971.4446 - val_loss: 5227.7388 - val_mean_squared_error: 5227.7388
Epoch 98/1000
1/1 [==============================] - 0s 271ms/step - loss: 2962.9487 - mean_squared_error: 2962.9487 - val_loss: 5213.1958 - val_mean_squared_error: 5213.1958
Epoch 99/1000
1/1 [==============================] - 0s 295ms/step - loss: 2951.4460 - mean_squared_error: 2951.4460 - val_loss: 5194.3486 - val_mean_squared_error: 5194.3486
Epoch 100/1000
1/1 [==============================] - 0s 306ms/step - loss: 2939.0474 - mean_squared_error: 2939.0474 - val_loss: 5178.3062 - val_mean_squared_error: 5178.3062
Epoch 101/1000
1/1 [==============================] - 0s 306ms/step - loss: 2928.1833 - mean_squared_error: 2928.1833 - val_

...

1/1 [==============================] - 0s 265ms/step - loss: 457.5091 - mean_squared_error: 457.5091 - val_loss: 1378.5342 - val_mean_squared_error: 1378.5342
Epoch 568/1000
1/1 [==============================] - 0s 274ms/step - loss: 455.9072 - mean_squared_error: 455.9072 - val_loss: 1374.3275 - val_mean_squared_error: 1374.3275
Epoch 569/1000
1/1 [==============================] - 0s 262ms/step - loss: 453.4211 - mean_squared_error: 453.4211 - val_loss: 1370.0771 - val_mean_squared_error: 1370.0771
Epoch 570/1000
1/1 [==============================] - 0s 266ms/step - loss: 451.1445 - mean_squared_error: 451.1445 - val_loss: 1364.2047 - val_mean_squared_error: 1364.2047
Epoch 571/1000
1/1 [==============================] - 0s 255ms/step - loss: 449.3262 - mean_squared_error: 449.3262 - val_loss: 1360.2125 - val_mean_squared_error: 1360.2125
Epoch 572/1000
1/1 [==============================] - 0s 262ms/step - loss: 447.4783 - mean_squared_error: 447.4783 - val_loss: 1355.2662 - val_m

Epoch 615/1000
1/1 [==============================] - 0s 310ms/step - loss: 375.7906 - mean_squared_error: 375.7906 - val_loss: 1168.0126 - val_mean_squared_error: 1168.0126
Epoch 616/1000
1/1 [==============================] - 0s 265ms/step - loss: 375.3476 - mean_squared_error: 375.3476 - val_loss: 1180.4320 - val_mean_squared_error: 1180.4320
Epoch 617/1000
1/1 [==============================] - 0s 313ms/step - loss: 385.9021 - mean_squared_error: 385.9021 - val_loss: 1158.4012 - val_mean_squared_error: 1158.4012
Epoch 618/1000
1/1 [==============================] - 0s 292ms/step - loss: 371.4139 - mean_squared_error: 371.4139 - val_loss: 1154.2655 - val_mean_squared_error: 1154.2655
Epoch 619/1000
1/1 [==============================] - 0s 315ms/step - loss: 369.2614 - mean_squared_error: 369.2614 - val_loss: 1150.2623 - val_mean_squared_error: 1150.2623
Epoch 620/1000
1/1 [==============================] - 0s 324ms/step - loss: 368.5689 - mean_squared_error: 368.5689 - val_loss: 11

Epoch 663/1000
1/1 [==============================] - 0s 303ms/step - loss: 324.2367 - mean_squared_error: 324.2367 - val_loss: 991.8812 - val_mean_squared_error: 991.8812
Epoch 664/1000
1/1 [==============================] - 0s 315ms/step - loss: 323.2768 - mean_squared_error: 323.2768 - val_loss: 988.6094 - val_mean_squared_error: 988.6094
Epoch 665/1000
1/1 [==============================] - 0s 277ms/step - loss: 322.3004 - mean_squared_error: 322.3004 - val_loss: 985.3877 - val_mean_squared_error: 985.3877
Epoch 666/1000
1/1 [==============================] - 0s 314ms/step - loss: 321.9867 - mean_squared_error: 321.9867 - val_loss: 982.2606 - val_mean_squared_error: 982.2606
Epoch 667/1000
1/1 [==============================] - 0s 267ms/step - loss: 320.6195 - mean_squared_error: 320.6195 - val_loss: 978.9349 - val_mean_squared_error: 978.9349
Epoch 668/1000
1/1 [==============================] - 0s 309ms/step - loss: 320.3444 - mean_squared_error: 320.3444 - val_loss: 975.8535 - v

Epoch 711/1000
1/1 [==============================] - 0s 284ms/step - loss: 300.1563 - mean_squared_error: 300.1563 - val_loss: 857.5080 - val_mean_squared_error: 857.5080
Epoch 712/1000
1/1 [==============================] - 0s 286ms/step - loss: 299.6280 - mean_squared_error: 299.6280 - val_loss: 855.3604 - val_mean_squared_error: 855.3604
Epoch 713/1000
1/1 [==============================] - 0s 324ms/step - loss: 300.2177 - mean_squared_error: 300.2177 - val_loss: 853.2032 - val_mean_squared_error: 853.2032
Epoch 714/1000
1/1 [==============================] - 0s 278ms/step - loss: 300.7599 - mean_squared_error: 300.7599 - val_loss: 850.5828 - val_mean_squared_error: 850.5828
Epoch 715/1000
1/1 [==============================] - 0s 303ms/step - loss: 298.8018 - mean_squared_error: 298.8018 - val_loss: 903.3925 - val_mean_squared_error: 903.3925
Epoch 716/1000
1/1 [==============================] - 0s 273ms/step - loss: 302.4505 - mean_squared_error: 302.4505 - val_loss: 844.8046 - v

Epoch 759/1000
1/1 [==============================] - 0s 325ms/step - loss: 273.0255 - mean_squared_error: 273.0255 - val_loss: 768.3240 - val_mean_squared_error: 768.3240
Epoch 760/1000
1/1 [==============================] - 0s 320ms/step - loss: 273.2197 - mean_squared_error: 273.2197 - val_loss: 761.2631 - val_mean_squared_error: 761.2631
Epoch 761/1000
1/1 [==============================] - 0s 317ms/step - loss: 274.7708 - mean_squared_error: 274.7708 - val_loss: 761.6115 - val_mean_squared_error: 761.6115
Epoch 762/1000
1/1 [==============================] - 0s 327ms/step - loss: 271.9456 - mean_squared_error: 271.9456 - val_loss: 758.4150 - val_mean_squared_error: 758.4150
Epoch 763/1000
1/1 [==============================] - 0s 318ms/step - loss: 267.6562 - mean_squared_error: 267.6562 - val_loss: 763.4534 - val_mean_squared_error: 763.4534
Epoch 764/1000
1/1 [==============================] - 0s 320ms/step - loss: 265.2496 - mean_squared_error: 265.2496 - val_loss: 751.6531 - v

In [17]:
predict = regressor.predict_generator(test_gen)

In [19]:
predict.shape

(250, 1)

In [23]:
predict[:10]

array([[66.516785],
       [66.52762 ],
       [66.538445],
       [66.48236 ],
       [66.5025  ],
       [66.51744 ],
       [66.61221 ],
       [66.633224],
       [66.746895],
       [66.75373 ]], dtype=float32)

In [22]:
result = x_test[-250:]
result.shape

(250, 1)

In [24]:
result[:10]

array([[51.44],
       [51.78],
       [49.4 ],
       [50.76],
       [51.28],
       [54.36],
       [54.06],
       [57.36],
       [56.2 ],
       [57.16]])